This notebook shows how to plot the output datafiles generated by McStas simulation.
Two approaches are presented:
- using `McStasScript`
- using `matplotlib`

# `McStasScript` approach

In [ ]:
from mcstasscript.interface import instr, plotter, functions, reader

In [ ]:
# path to load McStas simulation data 
path_to_model = '/Users/celinedurniak/Documents/WorkESS/V20/V20Diffraction_ROOT_ASC_McStas/'

In [ ]:
# display info about sample
InstrReader = reader.McStas_file(path_to_model + '/V20_config6/V20_config6.instr')

In [ ]:
# check McStas instrument file for the type of sample
McStasInstrument = instr.McStas_instr('McStasInstrument')
InstrReader.add_to_instr(McStasInstrument)
McStasInstrument.print_components()

In [ ]:
McStasInstrument.print_component('NAK')

In [ ]:
# access data stored after McStas simulations
data_to_plot = functions.load_data(path_to_model+'/V20_config6/')

In [ ]:
# Create a dictionary containing filenames, shape of output data and x, y labels
dict_mcstas_files={}

for item in data_to_plot:
    file_key = item.metadata.info['filename'].rstrip()
    xlabel = item.metadata.info['xlabel'].rstrip()
    ylabel = item.metadata.info['ylabel'].rstrip()
    type_array = item.metadata.info['type']
    start = type_array.find('(') + 1
    end = type_array.find(')', start)
    
    if ',' in type_array[start:end]:
        nx_value, ny_value = type_array[start:end].split(',')
        
        dict_mcstas_files[file_key] = ((int(nx_value),
                                        int(ny_value)), 
                                       xlabel, 
                                       ylabel)
    else:
        dict_mcstas_files[file_key] = (int(type_array[start:end]), 
                                       xlabel, 
                                       ylabel)
        
dict_mcstas_files

In [ ]:
# plot all datafiles individually
for item in data_to_plot:
    plotter.make_sub_plot(item)

# `matplotlib` approach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
# Folder containing output of McStas simulation
result_folder = 'V20_config6'

In [ ]:
# plot 1D data generated by McStas simulation and plot using matplotlib
selected_filename = 'monitor_Hetube6.dat'
# quick check that we have 1D or 2D data
print(dict_mcstas_files[selected_filename])

fig, ax = plt.subplots()

x, y = np.genfromtxt(path_to_model + result_folder+ "/" + selected_filename, 
                     usecols=(0,1),unpack=True)
ax.plot(x, y, label=selected_filename)
ax.set_xlabel(dict_mcstas_files[selected_filename][1])
ax.set_ylabel(dict_mcstas_files[selected_filename][2])
ax.legend()

In [ ]:
# plot 2D - the number of line to read is hard-coded but could be accessed from the header of the file: '# type: array_2d(150, 150)'
selected_filename = 'monitor_tx_DENEX.dat'
print(dict_mcstas_files[selected_filename])
# check that we are dealing with 2D data and set the limit to read the first matrix only
data2d = np.genfromtxt(path_to_model
                       + result_folder + '/' 
                       + selected_filename, 
                       max_rows=dict_mcstas_files[selected_filename][0][0])

fig, ax = plt.subplots()
contf = ax.imshow(np.flip(data2d, 0), aspect='auto')
ax.set_xlabel(dict_mcstas_files[selected_filename][1])
ax.set_ylabel(dict_mcstas_files[selected_filename][2])
ax.set_title(selected_filename)
cbar = fig.colorbar(contf)